In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [12]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

df = spark.sql("SELECT * FROM LH_APM.mard_apm.t_pbi_cu_consumption_impact_mins LIMIT 1000")
display(df)
print(df.printSchema())

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 548c8a94-55c4-4331-a9c6-06afb855ce59)

root
 |-- Date: date (nullable = true)
 |-- Workspace_Id: string (nullable = true)
 |-- Workspace_name: string (nullable = true)
 |-- Item_Id: string (nullable = true)
 |-- Item_kind: string (nullable = true)
 |-- Item_name: string (nullable = true)
 |-- Operation_name: string (nullable = true)
 |-- Impact_mins: double (nullable = true)
 |-- CU_consumed: double (nullable = true)
 |-- CapacityID: string (nullable = true)

None


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Initialize Spark session (usually already available in Fabric notebooks)
spark = SparkSession.builder.getOrCreate()

# Read the table
df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")

# Update the Impact_mins column based on conditions
updated_df = df.withColumn(
    "Impact_mins",
    when(
        (col("Date") == "2025-11-05") & 
        (col("CapacityID") == "40EF7A5E-1383-4947-A015-D6C46AFB919B"),
        0
    ).otherwise(col("Impact_mins"))
)

# Write back to the table (this will overwrite the table)
updated_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("mard_apm.t_pbi_cu_consumption_impact_mins")

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 5, Finished, Available, Finished)

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.sql.types import *

# Read the table
df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")

# Show the schema to understand data types
print("Schema:")
df.printSchema()

# Show sample data
print("\nSample data:")
df.show(10, truncate=False)

# Check the data types more specifically
print("\nData types:")
for field in df.dtypes:
    print(f"{field[0]}: {field[1]}")

# Look at date format specifically
print("\nSample Date values:")
df.select("Date").distinct().show(10, truncate=False)

# Check for the specific CapacityID
print(f"\nRecords with CapacityID = '40EF7A5E-1383-4947-A015-D6C46AFB919B':")
specific_capacity_df = df.filter(col("CapacityID") == "40EF7A5E-1383-4947-A015-D6C46AFB919B")
specific_capacity_df.show(10, truncate=False)

# Check date range to see if 2025-09-15 exists
print("\nDate range in the data:")
df.select("Date").agg({"Date": "min"}).show()
df.select("Date").agg({"Date": "max"}).show()

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 6, Finished, Available, Finished)

Schema:
root
 |-- Date: date (nullable = true)
 |-- Workspace_Id: string (nullable = true)
 |-- Workspace_name: string (nullable = true)
 |-- Item_Id: string (nullable = true)
 |-- Item_kind: string (nullable = true)
 |-- Item_name: string (nullable = true)
 |-- Operation_name: string (nullable = true)
 |-- Impact_mins: double (nullable = true)
 |-- CU_consumed: double (nullable = true)
 |-- CapacityID: string (nullable = true)


Sample data:
+----------+------------------------------------+-------------------------------+------------------------------------+---------------+-------------------------------+--------------+------------------+------------------+------------------------------------+
|Date      |Workspace_Id                        |Workspace_name                 |Item_Id                             |Item_kind      |Item_name                      |Operation_name|Impact_mins       |CU_consumed       |CapacityID                          |
+----------+---------------------------

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, to_date, lit
from datetime import datetime

# Read the table
df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")

# Convert string to proper date for comparison
target_date = to_date(lit("2025-11-05"), "yyyy-MM-dd")

# Update the Impact_mins column based on conditions
updated_df = df.withColumn(
    "Impact_mins",
    when(
        (col("Date") == target_date) & 
        (col("CapacityID") == "11CE1374-106A-4A28-A165-8A164EEFCC18"),
        0
    ).otherwise(col("Impact_mins"))
)

# Write back to the table
updated_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("mard_apm.t_pbi_cu_consumption_impact_mins")

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 8, Finished, Available, Finished)

Single Date with Semantic Model ID and Operation Name

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit
from delta.tables import DeltaTable

# Initialize Spark session (if not already available)
spark = SparkSession.builder.appName("UpdateImpactMins").getOrCreate()

# Define the table path
table_path = "Tables/t_pbi_cu_consumption_impact_mins"
lakehouse_name = "LH_APM"
schema_name = "mard_apm"

# Read the current table
df = spark.read.table(f"{lakehouse_name}.{schema_name}.t_pbi_cu_consumption_impact_mins")

# Define filter conditions
date_condition = "2025-08-27"
capacity_id = "11CE1374-106A-4A28-A165-8A164EEFCC18"

print("=== BEFORE UPDATE ===")
# Show records that will be affected
before_df = df.filter(
    (col("Date") == date_condition) & 
    (col("CapacityID") == capacity_id) &
    (
        ((col("Item_Id") == "B7DBD61D-88EA-404A-A6D7-C35C74B91157") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "3544F254-D2D1-4358-8C59-6C7A5919B126") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "9CA6869E-01C6-435A-ADAD-410714A7A9F5") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "E2AECD3B-D23B-4A78-B748-AE5DBF505D83") & (col("Operation_name") == "Query"))
    )
).select("Date", "CapacityID", "Item_Id", "Operation_name", "Impact_mins", "Item_name")

print(f"Records to be updated: {before_df.count()}")
before_df.show(truncate=False)

# Create the update logic using CASE WHEN
updated_df = df.withColumn(
    "Impact_mins",
    when(
        (col("Date") == date_condition) & 
        (col("CapacityID") == capacity_id),
        when(
            (col("Item_Id") == "B7DBD61D-88EA-404A-A6D7-C35C74B91157") & 
            (col("Operation_name") == "XMLA Read Operation"),
            lit(230.2)
        ).when(
            (col("Item_Id") == "3544F254-D2D1-4358-8C59-6C7A5919B126") & 
            (col("Operation_name") == "XMLA Read Operation"),
            lit(168.85)
        ).when(
            (col("Item_Id") == "9CA6869E-01C6-435A-ADAD-410714A7A9F5") & 
            (col("Operation_name") == "XMLA Read Operation"),
            lit(30.7)
        ).when(
            (col("Item_Id") == "E2AECD3B-D23B-4A78-B748-AE5DBF505D83") & 
            (col("Operation_name") == "Query"),
            lit(1105.2)
        ).otherwise(col("Impact_mins"))
    ).otherwise(col("Impact_mins"))
)

# Write the updated dataframe back to the table
# Using overwrite mode - make sure you have proper backups
updated_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{lakehouse_name}.{schema_name}.t_pbi_cu_consumption_impact_mins")

print("\n=== UPDATE COMPLETED ===")

# Verify the changes
print("\n=== AFTER UPDATE ===")
# Read the updated table
updated_table_df = spark.read.table(f"{lakehouse_name}.{schema_name}.t_pbi_cu_consumption_impact_mins")

# Show the same records after update
after_df = updated_table_df.filter(
    (col("Date") == date_condition) & 
    (col("CapacityID") == capacity_id) &
    (
        ((col("Item_Id") == "B7DBD61D-88EA-404A-A6D7-C35C74B91157") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "3544F254-D2D1-4358-8C59-6C7A5919B126") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "9CA6869E-01C6-435A-ADAD-410714A7A9F5") & (col("Operation_name") == "XMLA Read Operation")) |
        ((col("Item_Id") == "E2AECD3B-D23B-4A78-B748-AE5DBF505D83") & (col("Operation_name") == "Query"))
    )
).select("Date", "CapacityID", "Item_Id", "Operation_name", "Impact_mins", "Item_name")

print(f"Records after update: {after_df.count()}")
after_df.show(truncate=False)

# Show a detailed comparison
print("\n=== DETAILED COMPARISON ===")
comparison_df = before_df.alias("before").join(
    after_df.alias("after"),
    ["Date", "CapacityID", "Item_Id", "Operation_name"],
    "full_outer"
).select(
    col("before.Date").alias("Date"),
    col("before.Item_Id").alias("Item_Id"),
    col("before.Operation_name").alias("Operation_name"),
    col("before.Impact_mins").alias("Before_Impact_mins"),
    col("after.Impact_mins").alias("After_Impact_mins"),
    (col("after.Impact_mins") - col("before.Impact_mins")).alias("Difference")
)

comparison_df.show(truncate=False)

StatementMeta(, fc2854a5-6f89-43f1-8e8f-e9ffd4cea4ac, 6, Finished, Available, Finished)

=== BEFORE UPDATE ===
Records to be updated: 4
+----------+------------------------------------+------------------------------------+-------------------+-----------+------------------------------+
|Date      |CapacityID                          |Item_Id                             |Operation_name     |Impact_mins|Item_name                     |
+----------+------------------------------------+------------------------------------+-------------------+-----------+------------------------------+
|2025-08-27|11CE1374-106A-4A28-A165-8A164EEFCC18|3544F254-D2D1-4358-8C59-6C7A5919B126|XMLA Read Operation|0.0        |MCR Project Reporting Flat_XLS|
|2025-08-27|11CE1374-106A-4A28-A165-8A164EEFCC18|9CA6869E-01C6-435A-ADAD-410714A7A9F5|XMLA Read Operation|0.0        |MCR Data Availability Report  |
|2025-08-27|11CE1374-106A-4A28-A165-8A164EEFCC18|B7DBD61D-88EA-404A-A6D7-C35C74B91157|XMLA Read Operation|0.0        |MCR Project Reporting Flat    |
|2025-08-27|11CE1374-106A-4A28-A165-8A164EEFCC18|E2AE

# Multiple Dates

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, to_date, lit
from datetime import datetime
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info("="*80)
logger.info("Starting Impact_mins update process")
logger.info("="*80)

# Define multiple target dates
target_dates = ["2025-11-12", "2025-11-18", "2025-11-21"]
target_capacity_id = "11CE1374-106A-4A28-A165-8A164EEFCC18"

logger.info(f"\nTarget dates to update: {target_dates}")
logger.info(f"Target CapacityID: {target_capacity_id}")

# Read the table
logger.info("\nReading table: mard_apm.t_pbi_cu_consumption_impact_mins")
df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")
initial_count = df.count()
logger.info(f"Total records in table: {initial_count}")

# Convert Date column to date type if it's not already
df = df.withColumn("Date", to_date(col("Date")))

# Count records that will be affected
affected_count = df.filter(
    (col("Date").isin(target_dates)) & 
    (col("CapacityID") == target_capacity_id)
).count()

logger.info(f"\nRecords that will be updated: {affected_count}")

if affected_count == 0:
    logger.warning("⚠ WARNING: No records match the criteria. No updates will be made.")
else:
    logger.info(f"✓ Found {affected_count} records to update")

# Update the Impact_mins column based on conditions
logger.info("\nUpdating Impact_mins column...")
updated_df = df.withColumn(
    "Impact_mins",
    when(
        (col("Date").isin(target_dates)) & 
        (col("CapacityID") == target_capacity_id),
        0
    ).otherwise(col("Impact_mins"))
)

# Verify the update
logger.info("\nVerifying updates...")
verification_df = updated_df.filter(
    (col("Date").isin(target_dates)) & 
    (col("CapacityID") == target_capacity_id)
)

logger.info("Sample of updated records:")
verification_df.select("Date", "CapacityID", "Impact_mins").show(20, truncate=False)

# Write back to the table
logger.info("\nWriting updated data back to table...")
try:
    updated_df.write \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("mard_apm.t_pbi_cu_consumption_impact_mins")
    
    logger.info("✓ Table updated successfully!")
    
    # Final verification
    final_df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")
    final_count = final_df.count()
    
    logger.info(f"\nFinal record count: {final_count}")
    logger.info(f"Records before: {initial_count}")
    logger.info(f"Records after: {final_count}")
    
except Exception as e:
    logger.error(f"✗ Error writing to table: {e}")
    raise

logger.info("\n" + "="*80)
logger.info("Process completed successfully!")
logger.info("="*80)

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 12, Finished, Available, Finished)

2025-12-09 07:06:10,696 - INFO - ================================================================================
2025-12-09 07:06:10,697 - INFO - Starting Impact_mins update process
2025-12-09 07:06:10,697 - INFO - ================================================================================
2025-12-09 07:06:10,698 - INFO - 
Target dates to update: ['2025-11-12', '2025-11-18', '2025-11-21']
2025-12-09 07:06:10,698 - INFO - Target CapacityID: 11CE1374-106A-4A28-A165-8A164EEFCC18
2025-12-09 07:06:10,707 - INFO - 
Reading table: mard_apm.t_pbi_cu_consumption_impact_mins
2025-12-09 07:06:16,159 - INFO - Total records in table: 246144
2025-12-09 07:06:27,215 - INFO - ✓ Table updated successfully!
2025-12-09 07:06:28,951 - INFO - 
Final record count: 246144
2025-12-09 07:06:28,952 - INFO - Records before: 246144
2025-12-09 07:06:28,953 - INFO - Records after: 246144
2025-12-09 07:06:28,953 - INFO - 
2025-12-09 07:06:28,954 - INFO - Process completed successfully!
2025-12-09 07:06:28,955 

+----------+------------------------------------+-----------+
|Date      |CapacityID                          |Impact_mins|
+----------+------------------------------------+-----------+
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-21|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-21|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-12|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-12|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-12|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-18|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-12|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11-21|11CE1374-106A-4A28-A165-8A164EEFCC18|0.0        |
|2025-11

# Date and Workspace ID

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, to_date, lit
from datetime import datetime

# Read the table
df = spark.table("mard_apm.t_pbi_cu_consumption_impact_mins")

# Convert string to proper date for comparison
target_date = to_date(lit("2025-11-25"), "yyyy-MM-dd")

# Update the Impact_mins column based on conditions
updated_df = df.withColumn(
    "Impact_mins",
    when(
        (col("Date") == target_date) & 
        (col("CapacityID") == "11CE1374-106A-4A28-A165-8A164EEFCC18") &
        (col("Workspace_name") == "iGPM MCR Dashboarding")
        ,
        0
    ).otherwise(col("Impact_mins"))
)

# Write back to the table
updated_df.write \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("mard_apm.t_pbi_cu_consumption_impact_mins")

StatementMeta(, e43e0f6b-bcf4-42aa-a21b-8e55a764f5fa, 17, Finished, Available, Finished)